### Qdrant Langchain integration

#### Setup
There are various modes of how to run Qdrant, and depending on the chosen one, there will be some subtle differences. The options include:

- Local mode, no server required
- Docker deployments
- Qdrant Cloud

```bash
pip install -qU langchain-qdrant

pip install -qU langchain-huggingface

pip install fastembed
```

In [1]:
##Load Embedding Model using GPU/CPU
##HuggingFaceEmbeddings: broad model support, GPU acceleration, full Transformers ecosystem.
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-large")

In [2]:
## FastEmbedEmbeddings: CPU‐optimized, lightweight, no heavy deps, quantized ONNX inference, ideal for large‐scale offline encoding.
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

embeddings = FastEmbedEmbeddings(model_name="thenlper/gte-large",parallel=0)

### Creating Langchain Document

In [3]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees Fahrenheit.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
]

#### Creating Qdrant Vectorstore via Langchain

In [4]:
from langchain_qdrant import QdrantVectorStore, RetrievalMode

vectorstorestore = QdrantVectorStore.from_documents(
    documents,
    embeddings,
    # url=url,
    prefer_grpc=True,
    # api_key=api_key,
    location=":memory:",
    collection_name="my_documents",
)

In [5]:
vectorstorestore

### Adding content to Vectorstore

In [6]:
document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

In [7]:
new_docs = [document_9,document_10]
vectorstorestore.add_documents(new_docs)

['d206519fcdce4214834914a8b04ef32e', '6ad78e6c41574668b601d62ce3c44921']

In [8]:
vectorstorestore

#### Similarity Search in Vectorstore

In [9]:
results = vectorstorestore.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy", k=2
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet', '_id': 'da7ebdb8e47e48b7a464b2c391e29f79', '_collection_name': 'my_documents'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet', '_id': 'ab22be7e983b462b99e2d131cfd7d963', '_collection_name': 'my_documents'}]


#### Updating Existing Collection from Qdrant Cloud

In [ ]:
qdrant_url = "QDRANT_URL"
qdrant_api_key = "QDRANT_SERVER_API_KEY"
qdrant_collection_name = "existing_collection_name"

vector_store = QdrantVectorStore.from_existing_collection(
    embedding = embeddings,
    collection_name=qdrant_collection_name,
    url=qdrant_url,
    retrieval_mode=RetrievalMode.HYBRID,
    api_key=qdrant_api_key
    )

In [ ]:
new_docs = [document_9,document_10]
vectorstorestore.add_documents(new_docs)

#### Hybrid Vector Search
To perform a hybrid search using dense and sparse vectors with score fusion,

- The retrieval_mode parameter should be set to RetrievalMode.HYBRID.
- A dense embeddings value should be provided to the embedding parameter.
- An implementation of the SparseEmbeddings interface using any sparse embeddings provider has to be provided as a value to the sparse_embedding parameter.
- Note that if you've added documents with the HYBRID mode, you can switch to any retrieval mode when searching, since both the dense and sparse vectors are available in the collection.

In [ ]:
documents_list = ["List of Langchain Documents"]
collection_name = "qdrant_collection_name"
qdrant_api_key = "Qdrant_API_KEY"

In [ ]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_qdrant import FastEmbedSparse, RetrievalMode, QdrantVectorStore

sparse_embeddings = FastEmbedSparse(
        model_name="Qdrant/bm42-all-minilm-l6-v2-attentions",
    )

embedding_model = FastEmbedEmbeddings(model_name="thenlper/gte-large",parallel=0)

vector_store = QdrantVectorStore.from_documents(
        documents = documents_list,
        sparse_embedding = sparse_embeddings,
        embedding = embedding_model,
        collection_name = collection_name,
        url = qdrant_url,
        retrieval_mode = RetrievalMode.HYBRID,
        api_key = qdrant_api_key
    )